The [Taylor-Green Vortex](https://en.wikipedia.org/wiki/Taylor%E2%80%93Green_vortex) in 2D is defined with initial conditions

$$ 
V_x = -U\sin(x)\cos(y),\ V_y = U\cos(x)\sin(y)
$$

Let's simulate the evolution of that flow. First we include the packages we need.

In [ ]:
using WaterLily
include("../examples/TwoD_plots.jl") # For unix/OSX systems
# include("..\\examples\\TwoD_plots.jl") # For Windows systems
gr(show = :ijulia)

Now write a function to set up a simulation with the given initial conditions.

In [ ]:
function TGVortex(;p=7,N=[1,10],Re=100)
    # Define vortex size, velocity, viscosity
    L = 2^p; U = 1; ν = U*L/Re

    # Taylor-Green-Vortex initial velocity field
    V = apply(L+2,L+2,2) do i,vx
        x,y = @. (vx-1.5)*2π/L          # scaled coordinates
        i==1 && return -U*sin(x)*cos(y) # V_x
        i==2 && return  U*cos(x)*sin(y) # V_y
    end

    # Initialize simulation
    c = ones(L+2,L+2,2)  # no immersed solids
    a = Flow(V,c,zeros(2),ν=ν)
    b = MultiLevelPoisson(c)
    return Simulation(U,L,a,b),L
end

Running the cell below evolves this initial condition over time. We visualize the vorticity of the flow by taking the curl of the velocity.

In [ ]:
sim,L = TGVortex(Re=100)
tprint = WaterLily.sim_time(sim)

nprint,Δprint = 75,0.05
for i ∈ 0:nprint
    tprint+=Δprint
    sim_step!(sim,tprint)
    @inside sim.flow.σ[I] = WaterLily.curl(3,I,sim.flow.u)*L
    flood(sim.flow.σ,clims=(-5,5),legend=false,border=:none)
end

This 2D flow just decays over time, with the decay rate set by the Reynolds number. 
[The 3D version of this flow](https://www.youtube.com/watch?v=5sTt0wRNHkM) evolves completely differently; the symmetry is broken and turbulence emgerges and tears up the large slow vortices into small eddies very quickly.